In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.utils.class_weight import compute_class_weight
import collections

# ---------------------------
# Configuration
# ---------------------------
IMAGE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 5  # You can change this to 1 for testing
DATASET_DIR = "seed data set"  # Update path if needed
use_resnet = True  # Set to False to use the custom CNN

# ---------------------------
# Load Dataset
# ---------------------------
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_DIR,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
class_names = dataset.class_names
NUM_CLASSES = len(class_names)

# Visualize Samples
plt.figure(figsize=(10, 10))
for images, labels in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

# ---------------------------
# Check Class Distribution
# ---------------------------
class_counts = collections.defaultdict(int)
for _, labels in dataset.unbatch():
    class_counts[class_names[labels.numpy()]] += 1

print("\n📊 Class distribution:")
for name, count in class_counts.items():
    print(f"{name}: {count}")

# ---------------------------
# Dataset Splits
# ---------------------------
def get_dataset_partitions_tf(dataset, train_split=0.8, val_split=0.1, test_split=0.1):
    dataset = dataset.unbatch()
    dataset = dataset.shuffle(1000, seed=42)
    total = tf.data.experimental.cardinality(dataset).numpy()

    train_size = int(train_split * total)
    val_size = int(val_split * total)

    train_ds = dataset.take(train_size).batch(BATCH_SIZE)
    val_ds = dataset.skip(train_size).take(val_size).batch(BATCH_SIZE)
    test_ds = dataset.skip(train_size + val_size).batch(BATCH_SIZE)

    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

# Prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# ---------------------------
# Preprocessing Layers
# ---------------------------
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0 / 255)
])

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

# ---------------------------
# Model Definitions
# ---------------------------
def build_custom_cnn():
    model = models.Sequential([
        layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
        resize_and_rescale,
        data_augmentation,
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

def build_resnet_model():
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)
    )
    base_model.trainable = False

    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    x = resize_and_rescale(inputs)
    x = data_augmentation(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

# ---------------------------
# Build & Compile Model
# ---------------------------
model = build_resnet_model() if use_resnet else build_custom_cnn()
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)
model.summary()

# ---------------------------
# Compute Class Weights
# ---------------------------
labels_list = []
for _, label in dataset.unbatch():
    labels_list.append(label.numpy())

weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels_list),
    y=labels_list
)
class_weights = dict(enumerate(weights))
print("\n⚖️ Class Weights:", class_weights)

# ---------------------------
# Train the Model
# ---------------------------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weights
)

# ---------------------------
# Plot Training Results
# ---------------------------
print("📊 Available keys in history:", history.history.keys())

acc = history.history.get('accuracy', [])
val_acc = history.history.get('val_accuracy', [])
loss = history.history.get('loss', [])
val_loss = history.history.get('val_loss', [])

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
if acc:
    plt.plot(range(len(acc)), acc, label='Training Accuracy')
if val_acc:
    plt.plot(range(len(val_acc)), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')

plt.subplot(1, 2, 2)
if loss:
    plt.plot(range(len(loss)), loss, label='Training Loss')
if val_loss:
    plt.plot(range(len(val_loss)), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Loss')
plt.show()

# ---------------------------
# Single Image Prediction
# ---------------------------
for images_batch, labels_batch in test_ds.take(1):
    image = images_batch[0].numpy().astype('uint8')
    label = labels_batch[0].numpy()
    plt.imshow(image)
    plt.title(f"Actual: {class_names[label]}")
    plt.axis("off")
    plt.show()

    prediction = model.predict(tf.expand_dims(images_batch[0], 0))
    print(f"Predicted: {class_names[np.argmax(prediction[0])]}\n")

# ---------------------------
# Multiple Predictions with Visualization
# ---------------------------
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

plt.figure(figsize=(12, 12))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predict(model, images[i])
        actual_class = class_names[labels[i]]
        plt.title(f"Actual: {actual_class}\nPredicted: {predicted_class}\nConfidence: {confidence}%")
        plt.axis("off")
plt.show()

# ---------------------------
# Save the Model
# ---------------------------
model_dir = "../models/"
os.makedirs(model_dir, exist_ok=True)
model_files = [f for f in os.listdir(model_dir) if f.split('.')[0].isdigit()]
model_versions = [int(f.split('.')[0]) for f in model_files if f.split('.')[0].isdigit()]
model_version = max(model_versions or [0]) + 1
model_path = os.path.join(model_dir, f"{model_version}.keras")
model.save(model_path)
print(f"✅ Model saved to: {model_path}")


Found 5513 files belonging to 5 classes.



📊 Class distribution:
Immature soybeans: 1125
Spotted soybeans: 1058
Skin-damaged soybeans: 1127
Broken soybeans: 1002
Intact soybeans: 1201


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 8, 8, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,850,629 (90.98 MB)

...


⚖️ Class Weights: {0: 1.1003992015968065, 1: 0.9800888888888889, 2: 0.9180682764363031, 3: 0.9783496007098491, 4: 1.0421550094517957}
Epoch 1/5
    173/Unknown 445s 2s/step - accuracy: 0.2112 - loss: 1.7169

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


173/173 ━━━━━━━━━━━━━━━━━━━━ 453s 2s/step - accuracy: 0.2112 - loss: 1.7164
Epoch 2/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 364s 2s/step - accuracy: 0.2156 - loss: 1.6104
Epoch 3/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 369s 2s/step - accuracy: 0.2164 - loss: 1.6087
Epoch 4/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 357s 2s/step - accuracy: 0.2091 - loss: 1.6102
Epoch 5/5
173/173 ━━━━━━━━━━━━━━━━━━━━ 346s 2s/step - accuracy: 0.2048 - loss: 1.6110
📊 Available keys in history: dict_keys(['accuracy', 'loss'])


<Figure size 1200x1200 with 0 Axes>

✅ Model saved to: ../models/4.keras


In [3]:
print(model.output_shape)  # Should be (None, 5) for 5 classes

(None, 5)


In [7]:
from tensorflow.keras.layers import Dense
Dense(5, activation='softmax')


<Dense name=dense_2, built=False>

In [13]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 8, 8, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,376,465 (92.99 MB)

 Trainable params: 262,917 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 525,836 (2.01 MB)